<a href="https://colab.research.google.com/github/subodh2708/deep-learning---tensorflow-2.0/blob/main/08_NLP_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP in Tensorflow

In [1]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

--2022-09-19 07:37:43--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-19 07:37:44 (79.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data('nlp_getting_started.zip')


--2022-09-19 07:37:48--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 2607:f8b0:4023:c0d::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-09-19 07:37:48 (98.6 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
 import pandas as pd
 train_df = pd.read_csv("train.csv")
 test_df = pd.read_csv("test.csv")
 train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df),len(test_df)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"target:{target}","(real disaster)" if target > 0 else  "(not a disaster)")
  print(f"Text\n {text} \n")
  print("---\n")

target:1 (real disaster)
Text
 Senator 'alarmed' by reports U.S. military families harassed: A U.S. Senator said on Wednesday he was alarmed byÛ_ http://t.co/sbILA2Yqjq 

---

target:0 (not a disaster)
Text
 A friend is like blood they are not beside us always. But they come out when we are wounded. 

---

target:0 (not a disaster)
Text
 Motors Hot Deals #452 &gt;&gt; http://t.co/ED32PBviO7 10x 27W 12V 24V LED Work Light FLOOD Lamp Tractor Truck SUV UTV AÛ_ http://t.co/IfM6v6480P 

---

target:1 (real disaster)
Text
 I-77 Mile Marker 31 to 40 South Mooresville  Iredell Vehicle Accident Congestion at 8/6 1:18 PM 

---

target:0 (not a disaster)
Text
 Also in a matter of weeks Amazon's going to be deluged with poorly written indie dystopian fiction about teens escaping from blood farms. 

---



## splitting the data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_sentences, val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                          train_df_shuffled['target'].to_numpy(),
                                                                          test_size = 0.1,
                                                                          random_state = 42)

## converting text into numbers

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens= False
                                    )

In [13]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
max_vocab_length = 10000
max_length = 15

In [15]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [16]:
text_vectorizer.adapt(train_sentences)

In [17]:
sample_sentence = "there is flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [18]:
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence} \
      \n Vectorized version:\n {text_vectorizer([random_sentence])}')

Original text:
 @gregorysanders @USDOT &amp; the stat of high auto deaths applies to children in a vehicle. I guess they can out run lightrail better than adult       
 Vectorized version:
 [[   1 7126   35    2 8118    6  347    1  401 6213    5  656    4    3
   869]]


In [19]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"number of words in vocab:{len(words_in_vocab)}")
print(f"top 5 word in vocab:{top_5_words}")
print(f"bottom 5 words in vocab:{bottom_5_words}")

number of words in vocab:10000
top 5 word in vocab:['', '[UNK]', 'the', 'a', 'in']
bottom 5 words in vocab:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating the Embedding using Embedding layer

In [20]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim = max_vocab_length,
                      output_dim = 128,
                      embeddings_initializer ='uniform',
                      input_length = max_length)
embedding

In [21]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
        \n Embedded Version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Kakeru Teduka: Bfore 70years of today in Hiroshima it's exploded the one atomic bomd. It is so sad day.http://t.co/8Vzl1ns2iO         
 Embedded Version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04024135,  0.02389551,  0.00561875, ..., -0.01874375,
          0.02381838,  0.03405904],
        [-0.00030456, -0.00090928, -0.04247667, ..., -0.03512832,
         -0.00044885, -0.03928898],
        [ 0.04024135,  0.02389551,  0.00561875, ..., -0.01874375,
          0.02381838,  0.03405904],
        ...,
        [ 0.0430737 ,  0.01216229, -0.01691316, ...,  0.00834533,
         -0.0393994 ,  0.01110981],
        [ 0.04024135,  0.02389551,  0.00561875, ..., -0.01874375,
          0.02381838,  0.03405904],
        [-0.02188735, -0.03541207,  0.04941286, ...,  0.00242574,
         -0.01215143,  0.01276126]]], dtype=float32)>

In [22]:
random_sentence ,sample_embed[0][0],sample_embed[0][0].shape

("Kakeru Teduka: Bfore 70years of today in Hiroshima it's exploded the one atomic bomd. It is so sad day.http://t.co/8Vzl1ns2iO",
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 4.02413495e-02,  2.38955058e-02,  5.61875105e-03, -1.16589181e-02,
         1.15596429e-02,  3.53085287e-02,  3.65860946e-02, -4.37245853e-02,
         6.73120096e-03,  3.16516645e-02,  2.80974992e-02,  8.40027258e-03,
         2.27053873e-02,  1.15899071e-02,  4.27299030e-02,  2.47745998e-02,
         4.27461378e-02,  4.80383746e-02, -3.27328816e-02, -4.05038595e-02,
        -4.38809991e-02, -4.65368740e-02, -2.75174621e-02,  3.32072638e-02,
        -2.93009765e-02,  7.09710270e-03, -1.59216151e-02, -3.20770591e-03,
         3.33524458e-02, -1.24979988e-02, -5.15554100e-03, -1.46001577e-03,
        -2.80264858e-02,  3.33873294e-02,  4.03233506e-02, -4.67091799e-02,
         4.82814796e-02, -8.05555657e-03,  3.00943963e-02,  3.27971466e-02,
        -5.14621660e-03,  1.93208568e-02, -4.60452102e-02,  4

## Modelling text Dataset 

## Model 0: Getting the baseline model

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [24]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f' baseline accuracy : {baseline_score *100:.2f}%')

 baseline accuracy : 79.27%


In [25]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## creating our evaluation function for our model experiments

In [26]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = 'weighted')
  model_results = {'accuracy':model_accuracy,
                   'precision':model_precision,
                   'recall':model_recall,
                   'f1':model_f1}

  return model_results


In [27]:
baseline_results = calculate_result(y_true=val_labels,
                                    y_pred = baseline_preds)
print(baseline_results)

{'accuracy': 0.7926509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


##Model 1: simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

In [29]:
SAVE_DIR = 'model_logs'

In [30]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype= tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs,outputs, name= 'model_1_dense')

In [31]:
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [32]:
model_1_history= model_1.fit(x = train_sentences,
                           y= train_labels,
                           epochs = 5,
                           validation_data = (val_sentences,val_labels),
                           callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                    experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220919-073751
Epoch 1/5
215/215 [==============================] - 8s 32ms/step - loss: 0.6089 - accuracy: 0.6999 - val_loss: 0.5341 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 6s 28ms/step - loss: 0.4425 - accuracy: 0.8199 - val_loss: 0.4672 - val_accuracy: 0.7900
Epoch 3/5
215/215 [==============================] - 6s 29ms/step - loss: 0.3488 - accuracy: 0.8599 - val_loss: 0.4541 - val_accuracy: 0.7953
Epoch 4/5
215/215 [==============================] - 6s 29ms/step - loss: 0.2864 - accuracy: 0.8918 - val_loss: 0.4636 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 5s 23ms/step - loss: 0.2406 - accuracy: 0.9070 - val_loss: 0.4782 - val_accuracy: 0.7835


In [33]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4782 - accuracy: 0.7835


[0.4781894087791443, 0.7834645509719849]

In [34]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0:10]

array([[0.38207236],
       [0.7870774 ],
       [0.9979148 ],
       [0.12356347],
       [0.16513965],
       [0.9466537 ],
       [0.90846825],
       [0.9927101 ],
       [0.9624891 ],
       [0.26540697]], dtype=float32)

In [35]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [36]:
model_1_results = calculate_result(y_true = val_labels,
                                   y_pred = model_1_preds)
model_1_results

{'accuracy': 0.7834645669291339,
 'precision': 0.7875988615317319,
 'recall': 0.7834645669291339,
 'f1': 0.7806382038884261}

In [37]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))


array([False, False, False, False])

## Visualising learned embeddings

In [38]:
word_in_vocab = text_vectorizer.get_vocabulary()
len(word_in_vocab) , words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [40]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [41]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [42]:
# Let's download the saved embeddings locally
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

## Recurrent Neural Networks

###Model 2 : LSTM

In [43]:
# Create LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
# x = layers.LSTM(64, activation="tanh", return_sequences=True)(x) # use return_sequences=True if you want to stack recurrent layers 
# print(f"After LSTM cell with return_sequences=True: {x.shape}")
x = layers.LSTM(64, activation="tanh")(x)
print(f"After LSTM cell: {x.shape}")
# x = layers.Dense(64, activation="relu")(x) # optional dense layer to have on top of LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

After embedding: (None, 15, 128)
After LSTM cell: (None, 64)


In [44]:
model_2.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [45]:
history_2 = model_2.fit(train_sentences,
                        train_labels,
                        epochs= 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                 experiment_name = "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220919-073825
Epoch 1/5
215/215 [==============================] - 12s 40ms/step - loss: 0.2237 - accuracy: 0.9186 - val_loss: 0.5693 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 11s 52ms/step - loss: 0.1553 - accuracy: 0.9444 - val_loss: 0.5703 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 11s 51ms/step - loss: 0.1256 - accuracy: 0.9527 - val_loss: 0.6544 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 13s 58ms/step - loss: 0.1055 - accuracy: 0.9604 - val_loss: 0.8108 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0829 - accuracy: 0.9685 - val_loss: 1.0127 - val_accuracy: 0.7795


In [46]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

((762, 1), array([[3.8973689e-03],
        [7.3917812e-01],
        [9.9982846e-01],
        [7.3850840e-02],
        [3.6796927e-04],
        [9.9958676e-01],
        [9.4815189e-01],
        [9.9988741e-01],
        [9.9981451e-01],
        [6.4409345e-01]], dtype=float32))

In [47]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [48]:
model_2_results = calculate_result(y_true= val_labels,
                                   y_pred = model_2_preds)
model_2_results

{'accuracy': 0.7795275590551181,
 'precision': 0.7800661437025787,
 'recall': 0.7795275590551181,
 'f1': 0.7782139444386276}

In [49]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 3: GRU

In [50]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype= 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation= 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs,outputs, name='model_3_GRU')

In [51]:
model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [52]:
history_3 = model_3.fit(train_sentences,
                        train_labels,
                        epochs =5,
                        validation_data =(val_sentences, val_labels),
                        callbacks= [create_tensorboard_callback(dir_name= SAVE_DIR,
                                                                experiment_name='model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220919-073951
Epoch 1/5
215/215 [==============================] - 12s 44ms/step - loss: 0.1542 - accuracy: 0.9410 - val_loss: 0.7809 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 9s 43ms/step - loss: 0.0815 - accuracy: 0.9699 - val_loss: 0.8738 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 12s 54ms/step - loss: 0.0656 - accuracy: 0.9717 - val_loss: 1.3138 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 7s 31ms/step - loss: 0.0591 - accuracy: 0.9753 - val_loss: 0.8473 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 6s 29ms/step - loss: 0.0508 - accuracy: 0.9765 - val_loss: 1.2365 - val_accuracy: 0.7769


In [53]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[1.2427121e-02],
       [5.5328488e-01],
       [9.9998009e-01],
       [1.9962582e-01],
       [1.2827456e-05],
       [9.9981934e-01],
       [4.7675082e-01],
       [9.9999166e-01],
       [9.9996793e-01],
       [6.7584580e-01]], dtype=float32)

In [54]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [55]:
model_3_results = calculate_result(y_true= val_labels,
                                   y_pred = model_3_preds)
model_3_results

{'accuracy': 0.7769028871391076,
 'precision': 0.7798819103655997,
 'recall': 0.7769028871391076,
 'f1': 0.7743498017353971}

### Model 4: Bidirectional RNN

In [56]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences= True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 = tf.keras.Model(inputs,outputs,name='model_4_bidirectional')

In [57]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [58]:
model_4.compile(loss= 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [59]:
history_4 = model_4.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data = (val_sentences,val_labels),
                        callbacks = [create_tensorboard_callback(dir_name= SAVE_DIR,
                                                                 experiment_name= 'model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20220919-074038
Epoch 1/5
215/215 [==============================] - 14s 45ms/step - loss: 0.1165 - accuracy: 0.9594 - val_loss: 0.8650 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 9s 43ms/step - loss: 0.0528 - accuracy: 0.9775 - val_loss: 1.0436 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 9s 42ms/step - loss: 0.0451 - accuracy: 0.9787 - val_loss: 1.2869 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 8s 39ms/step - loss: 0.0421 - accuracy: 0.9807 - val_loss: 1.4448 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 8s 36ms/step - loss: 0.0367 - accuracy: 0.9818 - val_loss: 1.5007 - val_accuracy: 0.7664


In [60]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[1.7756402e-02],
       [9.8840344e-01],
       [9.9998641e-01],
       [2.3124474e-01],
       [1.2495678e-05],
       [9.9972105e-01],
       [7.7009487e-01],
       [9.9999237e-01],
       [9.9998730e-01],
       [9.9688977e-01]], dtype=float32)

In [61]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [62]:
model_4_results = calculate_result(y_true= val_labels,
                                   y_pred = model_4_preds)
model_4_results

{'accuracy': 0.7664041994750657,
 'precision': 0.7663548384223465,
 'recall': 0.7664041994750657,
 'f1': 0.7653289711694428}

### Model 5:conv1D

In [63]:
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(['this is a test sentence']))
conv_1d = layers.Conv1D(filters = 32,
                        kernel_size = 5,
                        activation = 'relu',
                        padding = 'valid')
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

In [64]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype= tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters= 64, kernel_size=5,strides=1,activation= 'relu',padding= 'valid')(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation= 'sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name= 'model_5_Conv1D')

model_5.compile(loss= 'binary_crossentropy',
                optimizer= tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)              

In [65]:
history_5 = model_5.fit(train_sentences,
                        train_labels,
                        epochs = 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                 'Conv1D')])

Saving TensorBoard log files to: model_logs/Conv1D/20220919-074205
Epoch 1/5
215/215 [==============================] - 6s 24ms/step - loss: 0.1249 - accuracy: 0.9607 - val_loss: 0.9105 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0769 - accuracy: 0.9726 - val_loss: 1.0133 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 5s 25ms/step - loss: 0.0615 - accuracy: 0.9756 - val_loss: 1.0891 - val_accuracy: 0.7612
Epoch 4/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0592 - accuracy: 0.9747 - val_loss: 1.1460 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0520 - accuracy: 0.9769 - val_loss: 1.2249 - val_accuracy: 0.7690


In [66]:
model_5_pred_probs=model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[1.3253513e-01],
       [9.3644947e-01],
       [9.9991757e-01],
       [3.8441002e-02],
       [1.3110690e-07],
       [9.8685575e-01],
       [9.6606469e-01],
       [9.9996114e-01],
       [9.9999928e-01],
       [7.2389317e-01]], dtype=float32)

In [68]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [70]:
model_5_results= calculate_result(y_true= val_labels,
                                  y_pred = model_5_preds)
model_5_results

{'accuracy': 0.7690288713910761,
 'precision': 0.7711368167182066,
 'recall': 0.7690288713910761,
 'f1': 0.7666669868908192}